In [ ]:
#Import Libraries


from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable



In [ ]:
args={}
kwargs={}
args['batch_size']=4
args['test_batch_size']=10
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=True


In [ ]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
class Net_FC_MNIST(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net_FC_MNIST, self).__init__()
        self.fc1 = nn.Linear(28*28*1, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):

        x = x.view(-1, 28*28)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)

In [ ]:
class Net_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_CIFAR10, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 32*32*3)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


# New Section

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        #print(data.shape)
        #print(output.shape)
        #print(target.shape)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




In [ ]:
model = Net_FC_MNIST()
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.405309
Train Epoch: 1 [40/60000 (0%)]	Loss: 2.090269
Train Epoch: 1 [80/60000 (0%)]	Loss: 1.894278
Train Epoch: 1 [120/60000 (0%)]	Loss: 1.661230
Train Epoch: 1 [160/60000 (0%)]	Loss: 1.496488
Train Epoch: 1 [200/60000 (0%)]	Loss: 1.569621
Train Epoch: 1 [240/60000 (0%)]	Loss: 1.071917
Train Epoch: 1 [280/60000 (0%)]	Loss: 1.331274
Train Epoch: 1 [320/60000 (1%)]	Loss: 0.148562
Train Epoch: 1 [360/60000 (1%)]	Loss: 0.790161
Train Epoch: 1 [400/60000 (1%)]	Loss: 0.568821
Train Epoch: 1 [440/60000 (1%)]	Loss: 0.828588
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.085710
Train Epoch: 1 [520/60000 (1%)]	Loss: 0.353586
Train Epoch: 1 [560/60000 (1%)]	Loss: 1.120054
Train Epoch: 1 [600/60000 (1%)]	Loss: 0.567011
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.584706
Train Epoch: 1 [680/60000 (1%)]	Loss: 0.835799
Train Epoch: 1 [720/60000 (1%)]	Loss: 0.967217
Train Epoch: 1 [760/60000 (1%)]	Loss: 0.082939
Train Epoch: 1 [800/60000 (1%)]	Loss: 0.528504
Train Epoch: 1 [8

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Streaming output truncated to the last 5000 lines.
Train Epoch: 7 [40480/60000 (67%)]	Loss: 0.001159
Train Epoch: 7 [40520/60000 (68%)]	Loss: 0.000032
Train Epoch: 7 [40560/60000 (68%)]	Loss: 0.000234
Train Epoch: 7 [40600/60000 (68%)]	Loss: 0.000705
Train Epoch: 7 [40640/60000 (68%)]	Loss: 0.001003
Train Epoch: 7 [40680/60000 (68%)]	Loss: 0.028991
Train Epoch: 7 [40720/60000 (68%)]	Loss: 0.000006
Train Epoch: 7 [40760/60000 (68%)]	Loss: 0.002778
Train Epoch: 7 [40800/60000 (68%)]	Loss: 0.006827
Train Epoch: 7 [40840/60000 (68%)]	Loss: 0.000009
Train Epoch: 7 [40880/60000 (68%)]	Loss: 0.000891
Train Epoch: 7 [40920/60000 (68%)]	Loss: 0.001046
Train Epoch: 7 [40960/60000 (68%)]	Loss: 0.000050
Train Epoch: 7 [41000/60000 (68%)]	Loss: 0.000216
Train Epoch: 7 [41040/60000 (68%)]	Loss: 0.002550
Train Epoch: 7 [41080/60000 (68%)]	Loss: 0.000095
Train Epoch: 7 [41120/60000 (69%)]	Loss: 0.003914
Train Epoch: 7 [41160/60000 (69%)]	Loss: 0.000009
Train Epoch: 7 [41200/60000 (69%)]	Loss: 1.022961

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
model = Net_CIFAR10()
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()
    
#print(output.shape)
#print(target.shape)


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.362820
Train Epoch: 1 [40/50000 (0%)]	Loss: 2.374291
Train Epoch: 1 [80/50000 (0%)]	Loss: 2.389760
Train Epoch: 1 [120/50000 (0%)]	Loss: 2.359442
Train Epoch: 1 [160/50000 (0%)]	Loss: 2.458796
Train Epoch: 1 [200/50000 (0%)]	Loss: 2.600176
Train Epoch: 1 [240/50000 (0%)]	Loss: 2.812958
Train Epoch: 1 [280/50000 (1%)]	Loss: 2.069488
Train Epoch: 1 [320/50000 (1%)]	Loss: 2.425400
Train Epoch: 1 [360/50000 (1%)]	Loss: 2.233649
Train Epoch: 1 [400/50000 (1%)]	Loss: 2.211229
Train Epoch: 1 [440/50000 (1%)]	Loss: 2.065841
Train Epoch: 1 [480/50000 (1%)]	Loss: 2.730093
Train Epoch: 1 [520/50000 (1%)]	Loss: 2.041957
Train Epoch: 1 [560/50000 (1%)]	Loss: 1.587155
Train Epoch: 1 [600/50000 (1%)]	Loss: 2.050259
Train Epoch: 1 [640/50000 (1%)]	Loss: 1.831487
Train Epoch: 1 [680/50000 (1%)]	Loss: 2.384465
Train Epoch: 1 [720/50000 (1%)]	Loss: 2.392946
Train Epoch: 1 [760/50000 (2%)]	Loss: 1.871271
Train Epoch: 1 [800/50000 (2%)]	Loss: 1.569679
Train Epoch: 1 [8

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Streaming output truncated to the last 5000 lines.
Train Epoch: 7 [480/50000 (1%)]	Loss: 0.974613
Train Epoch: 7 [520/50000 (1%)]	Loss: 2.068570
Train Epoch: 7 [560/50000 (1%)]	Loss: 1.340955
Train Epoch: 7 [600/50000 (1%)]	Loss: 0.837868
Train Epoch: 7 [640/50000 (1%)]	Loss: 2.293754
Train Epoch: 7 [680/50000 (1%)]	Loss: 3.065191
Train Epoch: 7 [720/50000 (1%)]	Loss: 0.363902
Train Epoch: 7 [760/50000 (2%)]	Loss: 0.913248
Train Epoch: 7 [800/50000 (2%)]	Loss: 1.410635
Train Epoch: 7 [840/50000 (2%)]	Loss: 0.929118
Train Epoch: 7 [880/50000 (2%)]	Loss: 2.035747
Train Epoch: 7 [920/50000 (2%)]	Loss: 1.103278
Train Epoch: 7 [960/50000 (2%)]	Loss: 1.982287
Train Epoch: 7 [1000/50000 (2%)]	Loss: 1.339595
Train Epoch: 7 [1040/50000 (2%)]	Loss: 1.203106
Train Epoch: 7 [1080/50000 (2%)]	Loss: 2.628306
Train Epoch: 7 [1120/50000 (2%)]	Loss: 1.201712
Train Epoch: 7 [1160/50000 (2%)]	Loss: 0.948615
Train Epoch: 7 [1200/50000 (2%)]	Loss: 0.986718
Train Epoch: 7 [1240/50000 (2%)]	Loss: 0.748663
Tr

In [ ]:
32*32*3

3072